# Mini-Lab: Logistic Regression and SVMs

Group Members:
Thomas Pengilly, 
Quynh Chao,
Michael Weatherford,
Anish Patel

The following analysis performs logistic regression and SVM algorithms to classify ?????the property type or property price?????.  

## Data Manipulation?

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import data
data = pd.read_csv('C:\\Users\\mweather\\Desktop\\Classes\\Term 3\\Machine Learning I\\mini_lab_2\\Imputed_Dataset.csv')

In [3]:
#Create a price class attribute 
data.loc[(data['price_class'] == 'Low'),'price_class_final'] = 0
data.loc[(data['price_class'] == 'Average'),'price_class_final'] = 1
data.loc[(data['price_class'] == 'High'),'price_class_final'] = 2

#remove not used columns and nans
data = data.drop(columns=['lat', 'lon','id','start_date','end_date','created_on','property_type','l1','l2','l3','title','description'])
data = data.dropna()

#Check our results 
data['price_class_final'].value_counts()

2.0    158761
1.0    154039
0.0    150923
Name: price_class_final, dtype: int64

## Modeling
Create a logistic regression model and a support vector machine model for the classification task.  Use an 80/20 split.  Adjust the model parameters to improve their accuracy.  If the dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use.

# Logistic Regression
### Code Adpated from Dr. Jake Drew's EducationDataNC Notebook (See Below)
### https://github.com/jakemdrew/EducationDataNC/blob/master/2017/Models/2017ComparingSegregatedHighSchoolCampuses.ipynb

In [4]:
# x explanatory and y response variables
Y_SplitPrice = data['price_class_final'] 
Y = Y_SplitPrice

#Delete not used columns 
X_SplitPrice = data.drop(columns=['price_class','price_class_final','Unnamed: 0'])

#data info
X_SplitPrice.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463723 entries, 0 to 463722
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   rooms            463723 non-null  float64
 1   bedrooms         463723 non-null  float64
 2   bathrooms        463723 non-null  float64
 3   surface_total    463723 non-null  float64
 4   surface_covered  463723 non-null  float64
 5   price            463723 non-null  float64
 6   log_price        463723 non-null  float64
dtypes: float64(7)
memory usage: 28.3 MB


In [5]:
#test and training split
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=6)

In [14]:
from sklearn.model_selection import cross_validate

def EvaluateClassifierEstimator(classifierEstimator, X, y, cv):
    
    #Perform cross validation 
    from sklearn.model_selection import cross_val_predict
    predictions = cross_val_predict(classifierEstimator, X_SplitPrice, Y, cv=cv)
    
    #model evaluation 
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    
    #pass true test set values and predictions to classification_report
    classReport = classification_report(Y,predictions)
    confMat = confusion_matrix(Y,predictions)
    acc = accuracy_score(Y,predictions)
    
    print(classReport)
    print(confMat)
    print(acc)

In [10]:
#Logisitic regression 10-fold cross-validation 
from sklearn.linear_model import LogisticRegression
regEstimator = LogisticRegression()


parameters = { 'penalty':['l2']
              ,'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
              ,'class_weight': ['balanced']
              ,'random_state': [6]
              ,'solver': ['lbfgs']
              ,'max_iter':[100,500]
              ,'multi_class':['multinomial']
             }

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=regEstimator
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring='accuracy')

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_SplitPrice, Y)

Fitting 10 folds for each of 14 candidates, totalling 140 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  4.5min
[Parallel(n_jobs=8)]: Done 140 out of 140 | elapsed: 17.0min finished


GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=6, test_size=0.2, train_size=None),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=8,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'class_weight': ['balanced'], 'max_iter': [100, 500],
                         'multi_class': ['multinomial'], 'penalty': ['l2'],
                         'random_state

In [11]:
#Diplay the top model parameters
regGridSearch.best_estimator_

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='multinomial', n_jobs=None,
                   penalty='l2', random_state=6, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [13]:
EvaluateClassifierEstimator(classifierEst, X_SplitPrice, Y, 10)

              precision    recall  f1-score   support

         0.0       0.91      0.80      0.85    150923
         1.0       0.71      0.70      0.71    154039
         2.0       0.72      0.82      0.77    158761

    accuracy                           0.77    463723
   macro avg       0.78      0.77      0.78    463723
weighted avg       0.78      0.77      0.77    463723

[[121160  14758  15005]
 [ 11651 107979  34409]
 [   681  28594 129486]]
0.773360389715412
